# Textgenerierung

Auch in dieser Woche beschäftigen wir uns mit Twitter-Daten. Diesmal geht es jedoch nicht um Sentiment Analysis, sondern wir wollen ein Modell trainieren, das es uns erlaubt, Tweets in einem bestimmten Stil zu generieren.

Als Datengrundlage dienen uns Daten von http://www.trumptwitterarchive.com. Brendan Brown, der Betreiber der Seite hat sämtliche Tweets von Donald Trump seit Mai 2009 zusammengetragen. Da wir die Sprache des US-Präsidenten modellieren wollen, verwenden wir nur dessen eigene und keine Retweets.

Unser Modell wird auf der Ebene von Einzelzeichen arbeiten, zunächst wollen wir uns aber auf einer höheren Ebene einen Überblick über den Datensatz verschaffen.

## 1. Aufgabe: Überblick über den Datensatz
### 1.1 Datensatz einlesen
Lest den in der Datei ```all_tweets.json``` enthaltenen Datensatz in einen Pandas-Dataframe mit folgenden Spalten ein: ```created_at```, ```id```, ```text```. Die übrigen im JSON enthaltenen Felder können ignoriert werden.



In [ ]:
import pandas as pd
import json

tweets = #TODO

tweets

### 1.2 Jahr hinzufügen
Für unsere Auswertungen interessiert uns nur das Jahr, in dem der Tweet verfasst wurde, nicht das genaue Datum. Wir fügen daher dem Dataframe eine zusätzliche Spalte ```year``` hinzu. Hierbei kann zum Beispiel die Pandas-Funktion ```DatetimeIndex``` verwendet werden, um aus dem ```created_at``` String einen DatetimeIndex zu machen, auf dessen einzelne Felder (```year```, ```month```, ```day```, ...) dann mittels Punktoperator zugegriffen werden kann.

In [ ]:
tweets['year'] = # TODO Spalte füllen
tweets

### 1.3 Textlänge analysieren
Naturgemäß gibt es bei Tweets nur eine begrenzte Varianz, was die Textlänge angeht. Wir wollen uns dennoch anschauen, wie sich die Textlänge im Laufe der Jahre entwickelt hat.
Dazu fügen wir unserem Dataframe zunächst eine Spalte ```text_length``` hinzu, in der wir festhalten, welche Länge der jeweilige Tweet-Text hat.

**Hinweis**
Mittels ```apply``` lassen sich Funktionen auf Spalten des Dataframes mappen: ```df['new'] = df['old'].apply(lambda x : fancy_stuff(x))```

In [ ]:
tweets['text_length'] = # TODO Spalte füllen
tweets

Für einen groben Überblick schauen wir uns einige Kennzahlen zur Textlänge an. Dazu gruppieren wir nach ```year```und nutzen dann die ```describe```-Methode des Dataframes, wobei wir nur Spalten vom Typ ```numpy.number``` betrachten und daher der ```describe```-Methode eine entsprechende ```include```-Liste mitgeben.

In [ ]:
import numpy
# TODO

### 1.4. Top-Hashtags und -Mentions
Nachdem wir uns mit der Länge der Texte beschäftigt haben, wollen wir nun herausfinden, wen Donald Trump in seinen Tweets erwähnt und welche Themen er (hash)taggt. Dabei interessiert uns die Entwicklung über die Jahre.

Anbei ein Vorschlag bezüglich des Vorgehens:
Wir beginnen mit den Hashtags und verwenden zunächst einen kleinen Trick, um ein Dictionary zu erstellen, das für jedes Jahr einen "Sub-Dataframe" enthält. 

Aus diesen Dataframes extrahieren wir dann pro Jahr alle Tweettexte in Form eines einzelnen Strings. Dazu konkatenieren wir die ```text```-Felder der Dataframes per ```' '.join(frame['text']) for frame in ...```

Damit haben wir ein Dictionary, das pro Jahr alle konkatenierten Tweet-Texte enthält, aus denen wir dann die Hashtags extrahieren können. 
Hierbei machen wir uns noch keine allzu großen Gedanken über Normalisierung, sondern zerlegen die langen Texte einfach per ```split()``` in einzelne Tokens, aus denen wir dann die Hashtags herausfiltern. 
Um die Top-Hashtags in Erfahrung zu bringen, verwenden wir wieder ```Counter```.

In [ ]:
from collections import Counter

tweets_by_year = dict(list(tweets.groupby(['year'])))
texts_per_year = {year: # hier befinden sich alle Tweets des Jahres als langer String}
top_hashtags_per_year = {year: # hier fehlt ein Counter mit den Top-Hashtags des Jahres, extrahiert aus den Tweet-Texten}
top_hashtags_per_year

Als nächstes interessieren uns die Mentions. Wir können hier analog zu den Hashtags vorgehen.

In [ ]:
top_mentions_per_year = # TODO
top_mentions_per_year

### 1.5 Vokabular
Bevor wir endgültig auf die Ebene der Einzelzeichen herabsteigen, wollen wir uns das von Trump verwendete Vokabular genauer ansehen und dabei auch herausfinden, ob weitere Vorverarbeitungsschritte nötig sind.

Dazu erstellen wir uns zunächst eine Liste aller Tokens, die in den Dokumenten vorkommen. Wir gruppieren nicht mehr per Jahr, sondern gehen ganz simpel vor und konkatenieren alle Tweet-Texte in einen langen String, den wir dann in einzelne Terme splitten.

Gebt die 20 häufigsten Terme aus. Was fällt auf (insbesondere bei Betrachtung des hinteren Endes der Liste)?

In [ ]:
tweet_texts = ' '.join(tweets['text'])
tokens = tweet_texts.split();

# TODO Top-20 Terme

Wenn es euch auch sinnvoll erscheint, die Html-Escapes wieder rückgängig zu machen, könnt ihr das hier tun.

In [ ]:
import html
cleaned_tweet_texts = # TODO

### 1.6 Verwendete Zeichen
Zum Ende unserer Analysephase schauen wir uns noch an, welche Einzelzeichen in den Tweets vorkommen. Dazu greifen wir wieder auf die konkatenierten (und bereinigten) Tweettexte zurück, die in der Variable ```cleaned_tweet_texts``` gespeichert sind.
Wie viele Zeichen gibt es und wie häufig werden sie verwendet?

In [ ]:
tweet_chars = # TODO Tweets als Liste von Zeichen (inklusive Duplikate). "Das hier sind alle Tweets" => ['D', 'a', 's', ' ', 'h', 'i', 'e', ....] (String => Liste)
# TODO: Welche Zeichen kommen vor?
# TODO: Wie oft wird welches Zeichen verwendet?

### 1.7 Vom Text zur Zeichenliste
Betrachtet die in der vorherigen Teilaufgabe generierte Liste der verwendeten Zeichen. Welche Bereinigungsschritte könnten angemessen sein? Kurze Begründung bitte.

Bevor wir daran gehen, vektorisierte Trainingsdaten für unser Modell zu generieren, wandeln wir unsere Tweets in dieser Teilaufgabe in eine (bereinigte) Liste von Einzelzeichen um und speichern diese in der Variable ```cleaned_tweet_chars```.

```['d', 'i', 'e', 's', ' ', 'i', 's', 't', ' ', 'e', 'i', 'n', ' ', 'b', 'e', 'i', 's', 'p', 'i', 'e', 'l', ',', ' ', 'w', 'i', 'e', ' ', 'd', 'i', 'e', ' ', 'l', 'i', 's', 't', 'e', ' ', 'b', 'e', 'g', 'i', 'n', 'n', 'e', 'n', ' ', 'k', 'ö', 'n', 'n', 't', 'e', '.']```


**Hinweise**: 
* In Python gibt es eine Methode ```str.isprintable()```, die bei der Bereinigung hilfreich sein könnte ...
* Vereinfachung ist legitim. Je mehr Einzelzeichen wir bei der Modellierung berücksichtigen, umso komplexer wird unser Modell und umso höher ist der Trainingsaufwand.

In [ ]:
cleaned_tweet_chars = # TODO Alle Tweets in Einzelzeichen zerlegt und ggf. bereinigt hier rein

## Aufgabe 2
Nachdem wir uns einen Überblick über den Datensatz verschafft und uns für die Zeichen entschieden haben, die wir bei der Modellierung berücksichtigen wollen, geht es nun darum, die Daten so aufzubereiten, dass wir ein Modell trainieren können.

Das Training soll wie folgt ablaufen: Gegeben n Zeichen, soll das Modell das n+1-te Zeichen vorhersagen. Dazu müssen wir die einzelnen Zeichen in Vektorform bringen. Wir wählen dazu ein One-Hot-Encoding und bilden folglich jedes der Einzelzeichen in ```cleaned_tweet_chars``` auf einen Vektor ab, der genau eine 1 enthält.

(Zu) einfaches Beispiel: ```['a', 'b', 'c'] => [(1,0,0), (0,1,0), (0,0,1)]```

### 2.1 Indizes für das One-Hot-Encoding
Um entscheiden zu können, an welcher Stelle wir die 1 setzen, müssen wir jedem Zeichen einen eindeutigen Index zuweisen.
Umgekehrt wollen wir auch zu jedem Index schnell das zugehörige Zeichen ermitteln können. Wir erstellen daher zwei Dictionaries: ```char2index``` für die Abbildung von Zeichen zu Index und ```index2char``` für die umgekehrte Abbildung von Index zu Zeichen.

In [ ]:
char_set = # TODO Menge aller verwendeten Zeichen nach Bereinigung
print('Anzahl Zeichen: {}'.format(len(char_set)))
char2index = # TODO Abbildung Zeichen => Index
index2char = # TODO Abbildung Index => Zeichen

### 2.2 Generiere Trainingsdaten
Wie bereits beschrieben, soll das Modell n Zeichen entgegennehmen und das n+1-te Zeichen vorhersagen. Wir generieren uns Trainingsdaten, indem wir ```sentences``` eine Liste von ```input_length``` Zeichen aus ```cleaned_tweet_chars``` hinzufügen, ```next_chars``` das darauf folgende ```input_length+1```-te Zeichen und dies alle ```step``` Zeichen wiederholen.

In [ ]:
input_length = 30
step = 4
sentences = []
next_chars = []

for i in range(# TODO start, ende, step):
    sentences.append(cleaned_tweet_chars[# TODO])
    next_chars.append(cleaned_tweet_chars[# TODO])
print('Anzahl Trainingssätze: {}'.format(len(sentences)))

### 2.3 Vektorisierung
Nun können wir daran gehen, die Daten zu vektorisieren. Die Eingabe ```x``` enthält für jeden der Trainingssätze in ```sentences``` one-hot-codierte Vektoren der Zeichen, aus denen der jeweilige Satz besteht. 
Die erwartete Ausgabe ```y``` basiert auf ```next_chars``` und enthält für jeden der Trainingssätze einen einzelnen one-hot-codierten Vektor für das als Fortsetzung des Satzes erwartete Zeichen.

Beispiel: Wenn ```sentences``` an Position ```i``` die Sequenz ```['a', 'b', 'c']``` und ```next_chars``` an derselben Stelle ```'d'``` enthält, dann enthält ```x``` bei entsprechender Kodierung an Position ```i``` ```[[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0]]``` und ```y``` an der entsprechenden Stelle ```[0, 0, 0, 1]```.

In [ ]:
import numpy as np

print('Vektorisierung ...')
x = np.zeros((len(sentences), input_length, len(char_set)), dtype=np.bool)
y = np.zeros((len(sentences), len(char_set)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[# TODO an welcher Position muss die 1 gesetzt werden?] = 1
    y[# TODO an welcher Position muss die 1 gesetzt werden] = 1

### 2.4 Definition des Modells

Für unseren Tweet-Generator werden wir ein recht einfaches Modell trainieren. Wir verwenden wieder die Sequential-API. 

Der erste Layer ist direkt das Herzstück unseres Modells: Der LSTM-Layer. 

Als Anzahl der Units verwenden wir 128.
Welche Dimensionen hat die ```input_shape```? 

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras import metrics

print('Erstelle Model...')
model = Sequential()
model.add(LSTM(128, input_shape=(input_length, # TODO zweite Dimension?)))

Als Ausgabelayer fügen wir einen Dense-Layer hinzu. Wie müssen wir die Anzahl der Hidden-Units wählen? Wieso ist ```softmax``` eine geeignete Aktivierungsfunktion?

In [ ]:
model.add(Dense(# TODO Anz. Units? , activation = 'softmax'))

Als Optimizer wählen wir RMSprop mit einer Learning-Rate von 0.005 und als Loss-Funktion ```categorical_crossentropy```.

Wieso können wir nicht wie im letzten Labor ```binary_crossentropy``` verwenden?

In [ ]:
optimizer = RMSprop(lr=0.005)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

print(model.summary())

### 2.5 Training des Models

Nach all den Vorarbeiten können wir nun daran gehen, unser Model zu trainieren.
Um das Model bei Bedarf nicht neu definieren zu müssen, speichern wir uns dessen Struktur als JSON ab.

In [ ]:
model_structure = model.to_json()
with open("text_generation_model.json", "w") as json_file:
    json_file.write(model_structure)

Um unsere Trainingsfortschritte nicht zu verlieren, definieren wir uns eine Checkpoint-Funktion, die als Callback aufgerufen wird und den aktuellen Modelzustand abspeichert, solange das Modell besser ist, als das bisher gespeicherte Model. Gespeichert werden soll das komplette Model, nicht nur die Gewichte.

In [ ]:
from keras.callbacks import ModelCheckpoint
model_checkpoint = ModelCheckpoint('text_generation.hd5', monitor='acc', # TODO nur bestes Model speichern, komplettes Model speichern)

Wir können unser Model nun trainieren. Um in ansehbarer Zeit Ergebnisse zu sehen, führen wir unser Training über 20 Epochen mit eine Batch-Size von 100 durch.

Um die Entwicklung des Models später auswerten zu können, speichern wir uns die History.

In [ ]:
import pickle

epochs = 20
batch_size = 100


model_history = # TODO Model fit mit Trainingsdaten, angegebenen Parametern und checkpoint

with open("text_generation_history", 'wb') as hist_file:
    pickle.dump(model_history.history, hist_file)
print('History gespeichert.')

## Aufgabe 3
Zum Abschluss wollen wir noch etwas Spaß mit unserem Modell haben. Aus diesem Grund laden wir uns das gespeicherte (bisher) beste Model und verwenden es, um basierend auf einem "Seed" neue Tweets zu generieren.

In [ ]:
from keras.models import load_model
loaded_model = load_model('text_generation.hd5')

### Eine kleine Hilfsfunktion
Der Ausgabelayer unseres Models beschreibt eine Wahrscheinlichkeitsverteilung über alle möglichen Ausgabezeichen. Was wir tun wollen, ist anhand dieser Verteilung eine repräsentative Stichprobe zu ziehen. Ähnlich wie bei Markov-Ketten wählen wir als nächstes Zeichen nicht zwangsläufig das, mit der höchsten Auftrittswahrscheinlichkeit, denn wir wollen uns ja eine gewissen künstlerisch-kreative Freiheit erhalten, aber wir orientieren uns bei der Auswahl an der Auftrittswahrscheinlichkeit der potentiellen Folgetokens im gegebenen Kontext.

Die Hilfsmethode, die wir dazu verwenden, ist aus dem Keras-Tutorial zu LSTMs übernommen. Der Parameter ```temperature``` schärft die ursprüngliche Wahrscheinlichkeitsverteilung oder schwächt sie ab. Bei ```temperature > 1``` ist die Ausgabe diverser, allerdings potentiell auch konfuser, bei ```temperature < 1 ``` bleiben wir näher an den Originaltweets.

```multinominal(num_samples, probabilities_list, size``` zieht ```size```-mal ```num_samples``` Beispiele aus einer Verteilung, deren Eigenschaften durch ```probabilities_list``` beschrieben wird. In unserem Fall wollen wir einmal ziehen und zwar genau ein Beispiel. Ausgabe ist demnach eine Liste, die einen einzigen Vektor enthält, der genauso lang ist, wie die Wahrscheinlichkeitsverteilung, die wir in die Funktion hineingeben, und der eine einzige 1 enthält. Für den Index dieser 1 interessieren wir uns, weil wir das entsprechend one-hot-codierte Zeichen als nächstes ausgeben wollen.

In [ ]:
import random

def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds.clip(min=0.0001)) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

### 3.1 Tweets erzeugen
Um neue Tweets zu erzeugen, brauchen wir einen Seed, mit dem unser Model arbeiten kann. Wir machen es uns einfach und wählen einen zufälligen Startindex, ab dem ```input_length``` Zeichen aus unserer langen Tweet-Liste ```cleaned_tweet_chars``` herausgenommen werden.

Diese vektorisieren wir dann und füttern unser Model mit dem so entstandenen Vektor, um das nächste Zeichen vorherzusagen, das dann wiederum Teil des Seeds für die nächste Vorhersage ist.


In [ ]:
import sys

seed_start_index = # TODO Zufallszahl für Startindex innerhalb von cleaned_tweet_chars wählen

for diversity in [0.2, 0.5, 0.8, 1.0, 1.2]:
    print()
    print('----- Diversität:', diversity)

    generated = ''
    sentence = # Subliste der passenden Länge aus cleaned_tweet_charts extrahieren
    generated += ''.join(sentence)
    print('----- Erzeuge Tweet aus Seed: "' + ''.join(sentence) + '"\n')
    sys.stdout.write(generated)

    for i in range(250):
        x = np.zeros((1, #TODO 2. Dimension?, len(char_set)))
        for t, char in enumerate(sentence):
            x[0, t, # TODO Wo muss die 1 hin?] = 1.

        preds = loaded_model.predict(x, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = # TODO Welchem Zeichen entspricht der Index?

        generated += next_char
        sentence = # TODO sentence bleibt eine Liste von Zeichen, aber das erste fällt weg und next_char kommt am Ende neu hinzu

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()

### 3.2 Entwicklung von Loss und Accuracy
Um die Aufgabe abzuschließen, wollen wir noch einen Blick auf die Entwicklung von Loss und Accuracy über die Trainingsepochen werfen. Wir haben die "Geschichte" unseres Models in der Datei ```text_generation_history``` abgespeichert.

Nutzt ```pyplot```, um die Entwicklung von Accuracy und Loss grafisch darzustellen.

In [ ]:
import matplotlib.pyplot as plt

with open("text_generation_history", 'rb') as hist_file:
    history = pickle.load(hist_file)
    
plt.plot(# Entwicklung der Accuracy)
plt.title('Accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.show()

# TODO Loss plotten
